##분류예측문제
성능이 우수한 예측모형 구축위해 데이터 전처리, 피처엔지니어링, 분류 알고리즘, 초매개변수 최적화, 모형 앙상블
- 수험번호.csv 파일이 만들어지도록 코드제출
- 제출모형희 성능은 ROC-AUC평가지표
- predict_proba 예측, 종속변수 survived 열의 범주 1확률 예측
- 데이터 파일읽기 예제

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

df = sns.load_dataset('titanic')
X_train, X_test, y_train, y_test = train_test_split(df, df['survived'], test_size=0.2, random_state=42, stratify=df['survived'])
X_train = X_train.drop(['alive', 'survived'], axis=1)
X_test = X_test.drop(['alive', 'survived'], axis=1)

print(X_train.isna().sum())


pclass           0
sex              0
age            137
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           553
embark_town      2
alone            0
dtype: int64


In [ ]:
#분포 살펴보기
print('deck',X_train['deck'].value_counts())
print('embarked ',X_train['embarked'].value_counts())
print('embark_town',X_train['embark_town'].value_counts())

deck C    41
B    34
E    29
D    26
A    14
F    11
G     4
Name: deck, dtype: int64
embarked  S    516
C    139
Q     55
Name: embarked, dtype: int64
embark_town Southampton    516
Cherbourg      139
Queenstown      55
Name: embark_town, dtype: int64


In [ ]:
from seaborn.colors import xkcd_rgb
#결측치 어떻게 처리할지
missing = ['age']
for i in missing :
  X_train[i] = X_train[i].fillna(X_train[i].mean())
  X_test[i] = X_test[i].fillna(X_test[i].mean())
X_train['deck'] = X_train['deck'].fillna('C')
X_test['deck'] = X_test['deck'].fillna('C')

X_train['embarked'] = X_train['embarked'].fillna('S')
X_test['embarked'] = X_test['embarked'].fillna('S')

X_train['embark_town'] = X_train['embark_town'].fillna('Southampton')
X_test['embark_town'] = X_test['embark_town'].fillna('Southampton')




In [ ]:
#정리된거 확인가능
print(X_train.isna().sum())


pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alone          0
dtype: int64


In [ ]:
#2. 라벨인코딩
from sklearn.preprocessing import LabelEncoder
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
label = ['sex','embarked', 'class', 'who','adult_male','deck','embark_town','alone']
X_train[label] = X_train[label].apply(LabelEncoder().fit_transform)
X_test[label] = X_test[label].apply(LabelEncoder().fit_transform)

In [ ]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
#데이터 타입 변환, 더미
print(X_train.dtypes)

pclass           int64
sex              int64
age            float64
sibsp            int64
parch            int64
fare           float64
embarked         int64
class            int64
who              int64
adult_male       int64
deck             int64
embark_town      int64
alone            int64
dtype: object


In [ ]:
dtype = ['pclass','sex','class']
for i in X_train[dtype] :
  X_train[i] = X_train[i].astype('category')
for i in X_test[dtype] :
  X_test[i] = X_test[i].astype('category')


In [ ]:
# island 2면 1 아니면 0, 데이터를 넓게 펴서 이진형으로 바꿔주는걸 더미처리라고 한다
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
X_train.head()

,age,sibsp,parch,fare,embarked,who,adult_male,deck,embark_town,alone,pclass_1,pclass_2,pclass_3,sex_0,sex_1,class_0,class_1,class_2
692,29.807687,0,0,56.4958,2,1,1,2,2,1,0,0,1,0,1,0,0,1
481,29.807687,0,0,0.0000,2,1,1,2,2,1,0,1,0,0,1,0,1,0
527,29.807687,0,0,221.7792,2,1,1,2,2,1,1,0,0,0,1,1,0,0
855,18.000000,0,1,9.3500,2,2,0,2,2,0,0,0,1,1,0,0,0,1
801,31.000000,1,1,26.2500,2,2,0,2,2,0,0,1,0,1,0,0,1,0


In [ ]:
#4. v파생변수
#연속형인거 바꿔주기
X_train['age_qcut'] = pd.qcut(X_train['age'],5,labels = False)
X_test['age_qcut'] = pd.qcut(X_test['age'],5,labels=False)

In [ ]:
#스케일
#0과1사이로
from sklearn.preprocessing import MinMaxScaler
scaler = ['age','fare']
min = MinMaxScaler()
min.fit(X_train[scaler])

X_train[scaler] = min.transform(X_train[scaler])
X_test[scaler] = min.transform(X_test[scaler])

In [ ]:
print(X_train.head())

          age  sibsp  parch      fare  embarked  who  adult_male  deck  \
692  0.369285      0      0  0.110272         2    1           1     2   
481  0.369285      0      0  0.000000         2    1           1     2   
527  0.369285      0      0  0.432884         2    1           1     2   
855  0.220910      0      1  0.018250         2    2           0     2   
801  0.384267      1      1  0.051237         2    2           0     2   

     embark_town  alone  pclass_1  pclass_2  pclass_3  sex_0  sex_1  class_0  \
692            2      1         0         0         1      0      1        0   
481            2      1         0         1         0      0      1        0   
527            2      1         1         0         0      0      1        1   
855            2      0         0         0         1      1      0        0   
801            2      0         0         1         0      1      0        0   

     class_1  class_2  age_qcut  
692        0        1         2  
481   

In [ ]:
#데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.2, random_state=42, stratify=y_train)

print(X_train.shape)
print(X_valid.shape)

(569, 19)
(143, 19)


In [ ]:
#모형학습, 앙상블
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1.fit(X_train,y_train)
pred1 = pd.DataFrame(model1.predict_proba(X_valid))



In [ ]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier()
model2.fit(X_train,y_train)
pred2 = pd.DataFrame(model2.predict_proba(X_valid))

In [ ]:
from sklearn.ensemble import VotingClassifier

model3 = VotingClassifier(estimators = [('logistc',model1),('random',model2)],voting = 'soft')
model3.fit(X_train,y_train)
pred3 = pd.DataFrame(model3.predict_proba(X_valid))

In [ ]:
print(pred3)
#첫번째 열만 제출해야한다

            0         1
0    0.083043  0.916957
1    0.661500  0.338500
2    0.176603  0.823397
3    0.962248  0.037752
4    0.859228  0.140772
..        ...       ...
138  0.908729  0.091271
139  0.908324  0.091676
140  0.919785  0.080215
141  0.051754  0.948246
142  0.455406  0.544594

[143 rows x 2 columns]


In [ ]:
#9 모형 평가
from sklearn.metrics import roc_auc_score
print("로지스틱",roc_auc_score(y_valid,pred1.iloc[:,1]))
print("랜포",roc_auc_score(y_valid,pred2.iloc[:,1]))
print("보팅",roc_auc_score(y_valid,pred3.iloc[:,1]))

로지스틱 0.8594008264462811
랜포 0.8542355371900827
뵝 0.8653925619834711


In [ ]:
## 10.하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[50,100],'max_depth':[4,6]}
model5 = RandomForestClassifier()
clf = GridSearchCV(estimator = model5,param_grid=parameters, cv=3)
clf.fit(X_train,y_train)
print("최적의 파라미터",clf.best_params_)


최적의 파라미터 {'max_depth': 4, 'n_estimators': 50}


In [ ]:
### 파일저장
result = pd.DataFrame(model3.predict_proba(X_test))
result = result.iloc[:,1]
pd.DataFrame({'id':X_test.index,'result':result}).to_csv('00300.csv',index=False)

In [ ]:
##확인
check =  pd.read_csv('00300.csv')
check.head()

,id,result
0,565,0.087698
1,160,0.056274
2,553,0.076876
3,860,0.047123
4,241,0.770561
